<a href="https://colab.research.google.com/github/elimart12/-CRUD_Terminal_MySQL/blob/main/Presidente_Minimax_Apha_Beta_IDS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Implementación de IA que juegue el juego de cartas **Presidente**
(*más famosamente conocido como conocido como "Culo" o "Comemierda"*).

> Se utilizará algoritmo minimax con podado alpha-beta y búsqueda de profundización iterativa.


> Su usará baraja francesa 【♦♥♣♠】






1. Crear el juego Presidente (jugable por consola) (orientado a estados).
2. Implementar minimax con podado alpha-beta.
3. Refinar con búsqueda de profundización iterativa (para manejo de tiempos de búsqueda).

> Clases a crear:


1. Card
2. Deck
3. Player
4. Game Session
5. Minimax Solver



In [ ]:
class Card():

  symbols = ["♦", "♥", "♣", "♠"]
  numbers = ["2", "3", "4", "5", "6", "7", "8", "9", "10", "J", "Q", "K", "A"]
  value_mapper = {"A": 15, "J": 11, "Q": 12, "K": 13, "3": 14}

  def __init__(self, symbol, number):
    self.symbol = symbol
    self.number = number
    self.value = int(Card.value_mapper.get(number, number))

  def __str__(self):
    space = " " if len(self.number) == 1 else ""
    return f"【{self.number}{space}{self.symbol}】"

  def __repr__(self):
    return str(self)

In [ ]:
import random

class Deck():

  def __init__(self, cards=None):
    self.cards = cards if cards is not None else []

  def add_card(self, card):
    self.cards.append(card)

  def get_cards(self, pos):
    if self.is_empty(): return None
    pos = [i%len(self.cards) for i in pos]
    return [card for idx, card in enumerate(self.cards) if idx in pos]

  def pop_cards(self, pos):
    if self.is_empty(): return None
    pos = [i%len(self.cards) for i in pos]
    popped_cards = [card for idx, card in enumerate(self.cards) if idx in pos]
    self.cards = [card for idx, card in enumerate(self.cards) if idx not in pos]
    return popped_cards

  def is_empty(self):
    return len(self.cards) == 0

  def flush(self):
    self.cards = list()

  def shuffle(self):
    random.shuffle(self.cards)

  @staticmethod
  def create_full_deck():
    cards = []
    for symbol in Card.symbols:
      for number in Card.numbers:
        cards.append(Card(symbol, number))
    return Deck(cards)

  def __str__(self):
    return str(self.cards)

  def __repr__(self):
    return str(self)

In [ ]:
class Player():

  def __init__(self, name, type_, deck):
    self.name = name
    self.type_ = type_
    self.deck = deck

In [ ]:
from google.colab import output
import itertools
import copy

class PresidentGameSession():

  def __init__(self, n_players, player_names=None, player_types=None, hide_print=False, minimax_time=1):

    print("Initializing configuration variables...")
    self.hide_print = hide_print
    self.minimax_time = minimax_time

    print("Creating players...")
    self.n_players = n_players
    self.player_names = player_names if player_names is not None else [f"player_{i}" for i in range(n_players)]
    self.player_types = player_types if player_types is not None else ["human"] + [f"AI" for i in range(n_players-1)]
    self.players = []
    self.create_players()

    print("Dealing cards...")
    self.deal_cards()

    print("Creating game variables...")
    self.last_played_player = None
    self.curr_turn = 0
    self.mode = None
    self.discard_deck = Deck()
    self.curr_player = self.players[self.curr_turn]
    self.turn_offset = 0
    self.top_cards = None
    self.curr_turn = 0
    self.winners = []

    input("Press enter to start!")

    output.clear()

  def create_players(self):
    self.players = [Player(self.player_names[i], self.player_types[i], Deck()) for i in range(self.n_players)]

  def deal_cards(self):
    full_deck = Deck.create_full_deck()
    full_deck.shuffle()
    player_idx = 0
    while not full_deck.is_empty():
      card = full_deck.pop_cards([-1])[0]
      self.players[player_idx].deck.add_card(card)
      player_idx = (player_idx + 1)%self.n_players

  def get_mode_as_number(self, mode):
    if mode is None: return None
    elif mode == "single": return 1
    elif mode == "double": return 2
    elif mode == "triplet": return 3
    else: return 4

  def __get_available_combinations_by_mode(self, mode):
    options = []
    for comb in itertools.combinations(enumerate(self.curr_player.deck.cards), mode):
      if len(set([comb[i][1].value for i in range(mode)])) == 1:
        if self.top_cards is None or comb[0][1].value >= self.top_cards[0].value:
          options.append("-".join([str(comb[i][0]) for i in range(mode)]))
    return options

  def get_available_decisions(self):

    options = []

    # Singles
    if self.mode == "single" or self.top_cards is None:
      options += self.__get_available_combinations_by_mode(1)
    else:
      for idx, card in enumerate(self.curr_player.deck.cards):
        if card.value == 15 and card.symbol == "♥":
          options.append(str(idx))

    # Doubles
    if self.mode == "double" or self.top_cards is None:
      options += self.__get_available_combinations_by_mode(2)

    # Triples
    if self.mode == "triplet" or self.top_cards is None:
      options += self.__get_available_combinations_by_mode(3)

    # Quads
    if self.mode == "quad" or self.top_cards is None:
      options += self.__get_available_combinations_by_mode(4)

    if self.top_cards is not None:
      options += ["x"]

    return options

  def get_player_decision(self):

    while True:
      index = input("Choose a card or 'x' to skip:").lower()
      if index in self.get_available_decisions(): return index

  def get_computer_decision_randomly(self):

    decision = random.choice(self.get_available_decisions())
    input(f"Computer plays {decision}. Press enter to continue")
    return decision

  def get_computer_decision_minimax(self):

    solver = MinimaxSolver(self.curr_player.name)
    state = copy.deepcopy(self)
    state.hide_print = True
    decision = solver.solve(state, self.minimax_time)
    input(f"Computer plays {decision}. Press enter to continue")
    return decision

  def start_loop(self):

    while True:

      if not self.hide_print:

        if self.last_played_player == self.curr_player: print("+ ROUND CLEANED!")
        print(f"> Turn of player \t{self.curr_player.name}")
        print(f"> Mode is \t\t{self.mode}")
        print(f"> Top card is \t\t{self.top_cards}")
        print(f"> Last played :\t\t{self.last_played_player.name if self.last_played_player else None}")
        print("-"*40)
        print(self.curr_player.deck)
        print(self.get_available_decisions())

      if self.curr_player.type_ == "AI": decision = self.get_computer_decision_minimax()
      else: decision = self.get_player_decision()

      game_ended = self.play_turn(decision)

      if game_ended: break

    results_dict = self.get_winners_points()
    if not self.hide_print: print("The winners are:", results_dict)

  def get_winners_points(self):
    max_points = self.n_players - 1
    result_dict = {}
    for winner in self.winners:
      result_dict[winner.name] = max_points * 1000
      max_points -= 1
    return result_dict

  @staticmethod
  def raise_invalid_input_exception():
    raise Exception("Invalid selection")
    print("Invalid selection...")

  def play_turn(self, decision):

    # INPUT ACTION TREE

    if decision not in self.get_available_decisions():
      self.raise_invalid_input_exception()

    if decision == "x":
      pass

    else:

      selected_cards_idx = decision.split("-")

      if len(selected_cards_idx) == 1: self.mode = "single"
      elif len(selected_cards_idx) == 2: self.mode = "double"
      elif len(selected_cards_idx) == 3: self.mode = "triplet"
      elif len(selected_cards_idx) == 4: self.mode = "quad"

      selected_cards_idx = list(map(int, selected_cards_idx))

      played_AH = False

      popped_cards = self.curr_player.deck.pop_cards(selected_cards_idx)

      for card in popped_cards:

        self.discard_deck.add_card(card)
        if card.value == 15 and card.symbol == "♥":
          self.turn_offset -= 1
          played_AH = True

      if not played_AH and self.top_cards and card.value == self.top_cards[0].value:
        self.turn_offset += 1

      self.last_played_player = self.curr_player

    if not self.hide_print: output.clear()

    # EXITING CHECKING

    if self.curr_player.deck.is_empty():
      if not self.hide_print: print(f"{self.curr_player.name} wins and exits round!!")
      self.winners.append(self.curr_player)
      self.discard_deck.flush()
      self.turn_offset = 0
      self.mode = None

    if len(self.winners) == self.n_players - 1:
      self.winners += [player for player in self.players if player not in self.winners]
      return True

    # TURN ENDING

    if self.mode is None: self.top_cards = None
    else: self.top_cards = self.discard_deck.get_cards([-(1+i) for i in range(self.get_mode_as_number(self.mode))])
    self.curr_turn = (self.curr_turn + 1 + self.turn_offset)%self.n_players
    self.turn_offset = 0

    self.curr_player = self.players[self.curr_turn]
    while self.curr_player in self.winners:
      self.curr_turn = (self.curr_turn + 1)%self.n_players
      self.curr_player = self.players[self.curr_turn]

    if self.last_played_player == self.curr_player:
      self.discard_deck.flush()
      self.top_cards = None
      self.mode = None

    return False

  def is_terminal(self):
    return len(self.winners) == self.n_players

  def children(self):
    options = self.get_available_decisions()
    children = []
    for option in options:
      child = copy.deepcopy(self)
      child.play_turn(option)
      children.append((option, child))
    return children

  def heuristic(self, player_name):
    heur = 0
    for player in self.players:
      if player.name == player_name:
        heur -= len(player.deck.cards)
      else:
        heur += len(player.deck.cards)
    return heur

In [ ]:
import numpy as np
import time

class MinimaxSolver():

  def __init__(self, player_name):

    self.player_name = player_name
    self.time_start = None
    self.max_time = None

  def __maximize(self, state, alpha, beta, depth):

    if time.time() - self.time_start >= self.max_time:
      raise StopIteration("Out of time!")

    if state.is_terminal():
      return None, state.get_winners_points()[self.player_name]

    if depth <= 0:
      return None, state.heuristic(self.player_name)

    max_child, max_utility = None, -np.inf

    for option, child in state.children():

      if child.curr_player.name == self.player_name:
        _, utility = self.__maximize(child, alpha, beta, depth-1)
      else:
        _, utility = self.__minimize(child, alpha, beta, depth-1)

      if utility > max_utility:
        max_child, max_utility = option, utility

      if max_utility >= beta:
        break

      alpha = max(alpha, max_utility)

    return max_child, max_utility

  def __minimize(self, state, alpha, beta, depth):

    if time.time() - self.time_start >= self.max_time:
      raise StopIteration("Out of time!")

    if state.is_terminal():
      return None, state.get_winners_points()[self.player_name]

    if depth <= 0:
      return None, state.heuristic(self.player_name)

    min_child, min_utility = None, np.inf

    for option, child in state.children():

      if child.curr_player.name == self.player_name:
        _, utility = self.__maximize(child, alpha, beta, depth-1)
      else:
        _, utility = self.__minimize(child, alpha, beta, depth-1)

      if utility < min_utility:
        min_child, min_utility = option, utility

      if min_utility <= alpha:
        break

      beta = min(beta, min_utility)

    return min_child, min_utility

  def solve(self, state, max_time):

    self.time_start = time.time()
    self.max_time = max_time
    for depth in range(2, 10000):
      try:
        best_option, _ = self.__maximize(state, -np.inf, np.inf, depth)
      except StopIteration:
        break

    return best_option

In [ ]:
game = PresidentGameSession(4, minimax_time=2)
game.start_loop()

player_3 wins and exits round!!
The winners are: {'player_0': 3000, 'player_1': 2000, 'player_3': 1000, 'player_2': 0}
